<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import numpy as np 
import keras
from keras.layers import Conv2D,MaxPooling2D,Dropout,LeakyReLU,concatenate,ZeroPadding2D,BatchNormalization,Conv2DTranspose, GlobalAveragePooling2D
from keras.layers import Add
from keras.models import Model
from keras.layers import Input, Dense
import os 
import cv2 
from matplotlib.pyplot import imshow,title ,show
from keras.applications import Xception
from keras import backend as K
from keras.losses import binary_crossentropy 
from keras.callbacks import ModelCheckpoint,Callback
from keras.optimizers import Adam, SGD
from matplotlib import pyplot as plt
import pandas as pd

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout, UpSampling2D
from keras.models import Model
from keras import regularizers 
from keras.layers.normalization import BatchNormalization as bn
from keras.applications.xception import Xception
from keras.layers import LeakyReLU,Add,ZeroPadding2D
from keras.losses import binary_crossentropy
from keras import callbacks


In [ ]:
img_size = 768
model = UXception(input_shape=(img_size,img_size,3))

In [ ]:
! nvidia-smi

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
from scipy.spatial import distance
def dice_score(true,pred):
    return 1-(distance.dice(true.ravel(),pred.ravel()))

In [ ]:
def weighted_crossentropy(y_true, y_pred):

    y_pred_f = K.reshape(y_pred, (batch_size*img_deps*img_rows,3))
    y_true_f = K.reshape(y_true, (batch_size*img_deps*img_rows,2))

    soft_pred_f = K.softmax(y_pred_f)
    soft_pred_f = K.log(tf.clip_by_value(soft_pred_f, 1e-10, 1.0))

    neg = K.equal(y_true_f, K.zeros_like(y_true_f))
    neg_calculoss = tf.gather(soft_pred_f[:,0], tf.where(neg))

    pos1 = K.equal(y_true_f, K.ones_like(y_true_f))
    pos1_calculoss = tf.gather(soft_pred_f[:,1], tf.where(pos1))

    pos2 = K.equal(y_true_f, 2*K.ones_like(y_true_f))
    pos2_calculoss = tf.gather(soft_pred_f[:,2], tf.where(pos2))

    loss = -K.mean(tf.concat([0.78*neg_calculoss, 0.65*pos1_calculoss, 8.57*pos2_calculoss], 0))

    return loss

def dice_loss(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return (binary_crossentropy(y_true, y_pred)+dice_loss(y_true, y_pred))

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

In [ ]:
def attach_attention_module(net, attention_module):
    if attention_module == 'se_block': # SE_block
        net = se_block(net)
    elif attention_module == 'cbam_block': # CBAM_block
        net = cbam_block(net)
    else:
        raise Exception("'{}' is not supported attention module!".format(attention_module))

    return net

def se_block(input_feature, ratio=8):
    """Contains the implementation of Squeeze-and-Excitation(SE) block.
    As described in https://arxiv.org/abs/1709.01507.
    """
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature._keras_shape[channel_axis]

    se_feature = GlobalAveragePooling2D()(input_feature)
    se_feature = Reshape((1, 1, channel))(se_feature)
    assert se_feature._keras_shape[1:] == (1,1,channel)
    se_feature = Dense(channel // ratio,
                       activation='relu',
                       kernel_initializer='he_normal',
                       use_bias=True,
                       bias_initializer='zeros')(se_feature)
    assert se_feature._keras_shape[1:] == (1,1,channel//ratio)
    se_feature = Dense(channel,
                       activation='sigmoid',
                       kernel_initializer='he_normal',
                       use_bias=True,
                       bias_initializer='zeros')(se_feature)
    assert se_feature._keras_shape[1:] == (1,1,channel)
    if K.image_data_format() == 'channels_first':
        se_feature = Permute((3, 1, 2))(se_feature)

    se_feature = multiply([input_feature, se_feature])
    return se_feature

def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """
    
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature._keras_shape[channel_axis]
    
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    
    avg_pool = GlobalAveragePooling2D()(input_feature)    
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool._keras_shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool._keras_shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool._keras_shape[1:] == (1,1,channel)
    
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool._keras_shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool._keras_shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool._keras_shape[1:] == (1,1,channel)
    
    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    
    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature._keras_shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature._keras_shape[-1]
        cbam_feature = input_feature
    
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool._keras_shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool._keras_shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat._keras_shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat) 
    assert cbam_feature._keras_shape[-1] == 1
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
        
    return multiply([input_feature, cbam_feature])



def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    #x = attach_attention_module(x,attention_module='cbam_block')
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    x = attach_attention_module(x,attention_module='cbam_block')
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    #x = attach_attention_module(x,attention_module='cbam_block')
    return x

def UXception(input_shape=(None, None, 3),dropout_rate=0.5):

    backbone = Xception(input_shape=input_shape,weights='imagenet',include_top=False)
    input = backbone.input
    start_neurons = 16

    conv4 = backbone.layers[121].output
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(dropout_rate)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    conv3 = backbone.layers[31].output
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    conv2 = backbone.layers[21].output
    conv2 = ZeroPadding2D(((1,0),(1,0)))(conv2)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = backbone.layers[11].output
    conv1 = ZeroPadding2D(((3,0),(3,0)))(conv1)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(dropout_rate)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
    
    uconv0 = Dropout(dropout_rate/2)(uconv0)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
    model = Model(input, output_layer)
    model.name = 'u-xception'

    return model

In [ ]:
x_train=np.load('./Numpy/float_x_unet_train.npy', mmap_mode='r')
y_train=np.load('./Numpy/float_y_unet_train.npy', mmap_mode='r')

x_val=np.load('./Numpy/float_x_unet_val.npy', mmap_mode='r')
y_val=np.load('./Numpy/float_y_unet_val.npy', mmap_mode='r')

In [ ]:
print (x_train.shape, y_train.shape)
print (x_val.shape, y_val.shape)

x_train_2=np.zeros(x_train.shape, dtype='float16')
for idx, i in enumerate(x_train):
    x_train_2[idx]=i.astype('float16')/255
    print (idx)
print (x_train_2[0])

x_val_2=np.zeros(x_val.shape, dtype='float16')
for idx, i in enumerate(x_val):
    x_val_2[idx]=i.astype('float16')/255
    print (idx)
print (x_val_2[0])

y_train=y_train.astype('float16')/255
y_val=y_val.astype('float16')/255

y_train=np.expand_dims(y_train,axis=-1)
y_val=np.expand_dims(y_val,axis=-1)

y_train[y_train>0.]=1
y_val[y_val>0.]=1

In [ ]:
np.unique(y_val)

In [ ]:
img_size = 768
model = UXception(input_shape=(img_size,img_size,3))

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,CSVLogger,ModelCheckpoint

adam=Adam(lr=1e-4)
model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy',dice_coef])

model_checkpoint = ModelCheckpoint(
        os.path.join('./Weights/', 'UNET_attention_tb.hdf5'),
        monitor='val_loss', mode='min',save_best_only=True, verbose=1)

#Callback for reducing learning rate based on validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=4, verbose=1, min_lr=1e-6)

#Callback for storing logs of learning rate, loss and accuracy
csvlogger=CSVLogger('./CSV/logs_from_scratch_adam_tb.csv')

callbacks = [model_checkpoint,reduce_lr,csvlogger]

In [ ]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=150,verbose=1,callbacks=callbacks,batch_size=2, shuffle=True)

In [ ]:
model.load_weights('./Weights/segmentation-Copy1.hdf5')

In [ ]:
op2=model.predict(x_val_2,verbose=1,batch_size=2)

In [ ]:
len(x_val)

In [ ]:
for i in range(len(op2)):
#     print(np.unique(y_val[i]))
#     print(np.unique(np.argmax(op2[i],axis=-1)))
    print (i)
    plt.figure(dpi=250)
    plt.subplot(1,4,1)
    plt.imshow((y_val[i][:,:,0]*255).astype('uint8'))
#     plt.imshow((x_val[i][:,:,0]*255).astype('uint8'))
#     plt.subplot(1,4,2)
#     plt.imshow((y_val[i][:,:,0]*255).astype('uint8'))
#     plt.subplot(1,4,3)
#     plt.imshow(x_val[i][:,:,0])
    # plt.imshow(op2[i][:,:,1])
    plt.subplot(1,4,2)
    new_mask=op2[i][:,:,0]>0.5
    new_mask=(new_mask*255).astype('uint8')
    plt.imshow(new_mask)
#     plt.subplot(1,4,4)
#     plt.imshow(np.argmax(op2[i],axis=-1))
    plt.show()

In [ ]:
# i=25
tp=0
stack=[]

for i in range(len(op2)):
    if np.max(y_val[i])>0.5:
        stack.append(dice_score(y_val[i],np.where(op2[i][:,:,0]>0.5,1,0)))
        if np.max(op2[i,:,:,0])>0.5:
            tp=tp+1
    print (i)

In [ ]:
np.average(stack)

In [ ]:
y_pred=[]
for idx, i in enumerate(op2):
    if np.max(i)>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
    print (idx)
        
test=[]
for i in y_val:
    if np.max(i)>0:
        test.append(1)
    else:
        test.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(test, y_pred, target_names=['Healthy', 'Edema']))

In [ ]:
len(x_val)

# Test Set 1 

In [ ]:
df_test=pd.read_csv('./CSV/test.csv')
df_test.head()

## Positive samples

In [ ]:
x_test=np.load('./Numpy/x_test.npy')
y_test=np.load('./Numpy/y_test.npy')

In [ ]:
x_test_2=np.zeros(x_test.shape, dtype='float16')
for idx, i in enumerate(x_test):
    x_test_2[idx]=i.astype('float16')/255
    print (idx)
print (x_test_2[0])

In [ ]:
y_test=y_test.astype('float16')/255

In [ ]:
y_test=np.expand_dims(y_test,axis=-1)
y_test[y_test>0.]=1

In [ ]:
op2=model.predict(x_test_2,verbose=1,batch_size=2)

In [ ]:
tp=0
stack=[]

for i in range(len(op2)):
    if np.max(y_test[i])>0.5:
        stack.append(dice_score(y_test[i],np.where(op2[i][:,:,0]>0.5,1,0)))
        if np.max(op2[i,:,:,0])>0.5:
            tp=tp+1
    print (i)
    
print ("TP:", tp)
print ("Total Positives:", len(op2))
print ("Recall:", float(tp)/len(op2))

In [ ]:
np.average(stack)

## Negative samples

In [ ]:
filenames=list(df_test['filepaths'])
labels=list(df_test['labels'])

In [ ]:
y_pred=[]
test=labels[200:]
for idx, i in enumerate(filenames[200:]):
    print (idx)
    image=cv2.imread(i)
    image=cv2.resize(image, (768,768))
    image=image.astype('float16')/255
    pred=model.predict(np.expand_dims(image, axis=0), verbose=1)
    
    if np.max(pred[0])>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test, y_pred)